In [ ]:
from pyspark.sql import SparkSession

# Tạo SparkSession
spark = SparkSession.builder \
    .appName("MNIST Classification") \
    .getOrCreate()

In [ ]:
# Đọc dữ liệu CSV
data_path = "path/to/MNIST_CSV.csv"  # Cập nhật đường dẫn đến tệp dữ liệu
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Hiển thị thông tin tổng quan về dữ liệu
df.show(5)
df.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer

# Tạo VectorAssembler để gộp các đặc trưng thành một vector
feature_columns = df.columns[:-1]  # Các cột đặc trưng (trừ cột nhãn)
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_raw")
df = assembler.transform(df)

# Chuẩn hóa dữ liệu
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=False)
df = scaler.fit(df).transform(df)

# Mã hóa nhãn
indexer = StringIndexer(inputCol="label", outputCol="label_index")
df = indexer.fit(df).transform(df)

df.select("features", "label_index").show(5)

In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Khởi tạo mô hình Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label_index")

# Huấn luyện mô hình
model = lr.fit(train_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)

# Đánh giá mô hình
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Hiển thị một số kết quả dự đoán
predictions.select("label_index", "prediction", "probability").show(5)
